# Readme




## how to use

1. Unzip and Install requirements
```
dask==2022.4.1
numpy==1.22.3
pandas==1.4.2
notebook==6.4.2
```


2. [option 1] Start Jupyter Notebook in the same folder

```bash
pip install -r requirements.txt
jupyter notebook
```
3. [option 2] run python file
```bash
python analyse_purchases.py
```

### Quick-Start

- populate `FILEPATH` with the relative or absolute location of csv and run the Cells

### Configuration of parameters

    FILTER_THRESHOLD: Populate this variable to filter relavant observations, defaulted to 2
    SEGMENT_CATEGORIES: Populate this list with the column headers of the csv file on which you want to create segments(refer to functionalities in next section for more details)
    




## Functionalities and Design Choices
- segement columns: Populate SEGMENT_CATEGORIES with the column names on which you want the segments to be created.
  - for example if SEGMENT_CATEGORIES = ["country"], then you will see observations like 
    - users from canada
    - users from korea
  - for example if  SEGMENT_CATEGORIES = ["country"], then you would see observation like 
    - users who are from canada
    - users who are from canada and are VIP
    - users who are from canada and are not VIP
- Segments are grouped, they will be ordered by the sequence of SEGMENT_CATEGORIES
  - for example if it is set to  ["country", "is_vip"] then it will be first grouped by country and then VIP status
    - For example all segments belonging to CANADA will occur together
      - users who are from canada
      - users who are from canada and are VIP
      - users who are from canada and are not VIP
<!-- - Ignoring Users: a function called user_type add a column called `user_type` which categorizes the users into 4 categories: ["regular_customer","non_paying","abstained_yesterday","abstained_today"]. A variable IGNORE_USER_TYPES is populated and all user types in this list would be excluded from the analysis. -->
- `getDf()` Have created this function so that if in future if we need to read from some other source such as mysql, we can just edit this function
- `metric` have tried to maintain this variable whenever possible so that in future in we have a new metric we can easily add more conditions

## Assumptions
- percentage calculation logic: The percentage is being calculated by comparing to yesterday
$$
 PercentChange = \frac{PurchaseToday - PurchaseYesterday}{PurchaseYesterday}\times 100
  $$
- if the amount for a segment was 0 yesterday and non zero today then that is considered as 100% increase
- if the amount for a segement was non zero yesterday and 0 today then that is considered as 100% drop
- Ignore users on each day who did not purchase any amount. An user can be included in yesterday's analysis and not be included in today's analysis
- User_ids are not repeated: if repeated we can first groupBy-Sum on user_id,country and is_vip and then start the whole process



## Future steps

- Reusability: add functions so that if business logic changes, the amount of changes we need to do is minimal
- Parallelization: Very similar to apache spark we can use dask in an asynchronus manner, so that it computes only when results are needed. The current parallelization is close to none

In [1]:
!pip install 'fsspec>=0.3.3'
!pip install dask

In [2]:
import pandas as pd
import numpy as np
import dask.dataframe as dd  # Dask Multiprocessing
from itertools import combinations
import math


# Reader and Filter Module

In [3]:


def getDf(filename):
    """
    function to return data frame from csv,
    can be modified to read from mysql
    """
    df = pd.read_csv(filename)
    return df


def user_type(yesterday, today):
    """
    function to find out the types of users
    """
    if (yesterday == 0 and today == 0):
        return "non_paying"
    if (yesterday == 0 and today != 0):
        return "abstained_yesterday"
    if (yesterday != 0 and today == 0):
        return "abstained_today"
    if (yesterday != 0 and today != 0):
        return "regular_user"


def amount_change(yesterday, today):
    return today - yesterday


# Stats Generation and Analysis Module


In [4]:
def segment_name(row, columns):
    values = [column + ":" + str(row[column]) for column in columns]
    return ','.join(values)


def getCombinations(columns):
    n = len(columns)
    res = [list(com) for sub in range(n) for com in combinations(columns, sub + 1)]
    return res


def getChangePercentage(today,yesterday):
    if(today == yesterday): # handles if both values are 0
        return 0
    elif(yesterday == 0):
        return 100
    elif(math.isnan(today)):
      return -100
    elif(math.isnan(yesterday)):
      return 100
    else:
        return (today-yesterday)*100/yesterday

def get_analysis(df, metric, segment_columns, metric_column_change_percentage, numerator_column, denominator_column):
    """
    Gets Segments on segment_columns and calculates stats(value and percentage) 
    on the given metric and returns an standard dataframe
    
    Arguments:
    df: input data frame
    metric: "total"/"average"
    segment_columns: "column header on which segments to be generated"
    numerator_column: numerator column which has today's value
    denominator_column: denominator column which has today's value
    """

    # get all posible combinations of given segment categories
    all_segments = getCombinations(segment_columns)

    combined_df = pd.DataFrame()
    for segment in all_segments:
        current_segment = df[df[numerator_column] > 0]
        old_segment = df[df[denominator_column] > 0]

        #  get metric for current and old column
        if (metric == "total"):
            current_segment = current_segment.groupby(segment)[numerator_column].sum().reset_index()
            old_segment = old_segment.groupby(segment)[denominator_column].sum().reset_index()
        elif (metric == "average"):
            current_segment = current_segment.groupby(segment)[numerator_column].mean().reset_index()
            old_segment = old_segment.groupby(segment)[denominator_column].mean().reset_index()

        # joining on segment
        segment_df = pd.merge(current_segment, old_segment, how="outer", on=segment)
        

        # get percentage
        segment_df[metric_column_change_percentage] = segment_df.apply(lambda x: getChangePercentage(x[numerator_column],x[denominator_column]), axis=1)
        segment_df["segment_name"] = segment_df.apply(lambda x: segment_name(x, segment), axis=1)

        # After outer join if NaN value is present for today it means it's 100% drop
        # NaN value for yesterday means it is 100 increase
        segment_df[numerator_column] = segment_df[numerator_column].replace(np.nan, -100)
        segment_df[denominator_column] = segment_df[denominator_column].replace(np.nan, 100)
        # selecting columns to maintain standard format
        segment_df = segment_df[['segment_name', numerator_column, metric_column_change_percentage]]
        combined_df = pd.concat([combined_df,segment_df])

    # sorting by segment_name to group segments together
    combined_df = combined_df.sort_values(by=['segment_name']).reset_index(drop=True)
    return combined_df




def get_overallStats(df,metric,numerator_column, denominator_column):
    """
    Returns overall absolute change, overall absolute change percentage
    and overall_purchase today for the given metric without segmenting
    """
    if(metric == "total"):
        overall_purchase_yesterday = df[denominator_column].sum()
        overall_purchase_today = df[numerator_column].sum()
    elif(metric == "average"):
        overall_purchase_yesterday = df[denominator_column].replace(0, np.nan).mean(skipna=True)
        overall_purchase_today = df[numerator_column].replace(0, np.nan).mean(skipna=True)

    if(math.isnan(overall_purchase_today)):
        overall_purchase_today = 0
    if(math.isnan(overall_purchase_yesterday)):
        overall_purchase_yesterday=0

    overall_absolute_change = overall_purchase_today-overall_purchase_yesterday
    overall_change_percentage = getChangePercentage(overall_purchase_today,overall_purchase_yesterday)
    return overall_purchase_today,overall_absolute_change, overall_change_percentage




# Human Readable String generation module

In [5]:
def map_category(key_value):
    """
    returns human readable segments string given segement_name in specific format
    example: "Users who are from australia and are VIP"
    """
    pair = key_value.split(":")
    c = ""
    if(pair[0] == "country"):
        c = "are from "+pair[1]
    else:
        if(pair[1] == "False"):
            c = "are not VIP"
        else:
            c = "are VIP"
    return c

def get_stats_string(amount,percentage):
    """
    returns human readable stats String given current amount and percentage
    example: "$2805633 (Down 212.58%  from yesterday)"
    """
    string = ': $%d' % amount
    if(percentage >= 0):
        string = string + " (Up %.2f%%  from yesterday)"%percentage
    else:
        string = string + " (Down %.2f%%  from yesterday)"%abs(percentage)
    return string

def humanReadableString(segment_name,current_amount,percentage_change):
    """
    returns Human readble string using:
    segment_name: segment's name in given format "country:australia,is_vip:True"
    Current_amount: Today's amount
    percentage_change: float number which represents change from yesterday
    Example output: "Users who are from australia and are VIP: $2805633 (Down 212.58%  from yesterday)"
    """
    categories = segment_name.split(",")
    final_string = "Users who "
    categories = [map_category(c) for c in categories]

    final_string = final_string + " and ".join(categories) + get_stats_string(current_amount,percentage_change)
    return final_string

# print(humanReadableString("country:australia,is_vip:True",2.805633e+06,0))


# Driver Module

In [6]:
def analyse_and_print_observations(df,metric,SEGMENT_CATEGORIES,NUMERATOR_COLUMN,DENOMINATOR_COLUMN,NUMERATOR_COLUMN_PERCENTAGE,FILTER_THRESHOLD):
    """
    Driver function to trigger analysis on given metric and print results
    Arguments:
    metric: "total" or "average"
    SEGMENT_CATEGORIES: "column header on which segments to be generated"\
    NUMERATOR_COLUMN: "numerator column header which has today's purchase"
    DENOMINATOR_COLUMN: "denominator column which holds value for last day"
    FILTER_THRESHOLD: "threshold percentage for which we show observations"
    """
    
    # Getting Analysis on segment level and then on overall level
    print("analysing for %s metric" % metric)
    metric_analysis_df = get_analysis(df, metric, SEGMENT_CATEGORIES, NUMERATOR_COLUMN_PERCENTAGE, NUMERATOR_COLUMN,
                                      DENOMINATOR_COLUMN)
    overall_absolute_value, overall_absolute_change, overall_change_percentage = get_overallStats(df,metric,NUMERATOR_COLUMN, DENOMINATOR_COLUMN)

    ## Creatig comparision columns for filtering 
    metric_analysis_df["value_change_wrt_overall"] = (metric_analysis_df[
                                                          NUMERATOR_COLUMN] * 100) / overall_absolute_change
    metric_analysis_df["value_change_wrt_overall"] = metric_analysis_df["value_change_wrt_overall"].abs()                                                      
    metric_analysis_df["percentage_change_wrt_overall"] = (metric_analysis_df[
                                                               NUMERATOR_COLUMN_PERCENTAGE] - overall_change_percentage)
    metric_analysis_df["percentage_change_wrt_overall"] = metric_analysis_df["percentage_change_wrt_overall"].abs()

    # filtering based on FILTER_THRESHOLD
    metric_analysis_df = metric_analysis_df[(metric_analysis_df['value_change_wrt_overall'] > FILTER_THRESHOLD) & (
            metric_analysis_df['percentage_change_wrt_overall'] > FILTER_THRESHOLD)]

    # getting results in Human readable string and printing them
    final_df = metric_analysis_df
    final_df['human_readable_observation'] = final_df.apply(
        lambda x: humanReadableString(x['segment_name'], x[NUMERATOR_COLUMN], x[NUMERATOR_COLUMN_PERCENTAGE]), axis=1)

    
    if (overall_change_percentage >= 0):
        print(metric.upper()+" purchases %d (Up %.2f%% from yesterday)" % (
        overall_absolute_value, overall_change_percentage))
    else:
        print(metric.upper()+" purchases %d (Down %.2f%% from yesterday)" % (
        overall_absolute_value, overall_change_percentage))

        
    results = final_df['human_readable_observation'].tolist()

    for result in results:
        print(result)
    return final_df



In [7]:
if __name__ == '__main__':

    # Location of csv file
    FILEPATH = "comparative.ai_take_home_test.csv"

    # Configuration of parameters
    FILTER_THRESHOLD = 2
    SEGMENT_CATEGORIES = ["country", "is_vip"]
    IGNORE_USER_TYPES = ["non_paying","abstained_yesterday","abstained_today"]

    # Detailed Configuration of parameters
    use_parallel_processing = True
    num_partitions = 12
    NUMERATOR_COLUMN = 'purchased_amount_today'
    DENOMINATOR_COLUMN = 'purchased_amount_yesterday'
    NUMERATOR_COLUMN_PERCENTAGE = 'amount_change_percentage'


    df = getDf(FILEPATH)
    print("Read CSV")
    # Creation of user_type column to represent the type of user in terms of:
    # ["regular_customer","non_paying","abstained_yesterday","abstained_today"]
    if use_parallel_processing == True:
        partitionedDf = dd.from_pandas(df, npartitions=num_partitions)
        df['user_type'] = partitionedDf.map_partitions(
            lambda df: df.apply(lambda x: user_type(x[DENOMINATOR_COLUMN], x[NUMERATOR_COLUMN]),
                                axis=1)).compute(scheduler='processes')
    else:
        df['user_type'] = df.apply(lambda x: user_type(x[DENOMINATOR_COLUMN], x[NUMERATOR_COLUMN]),
                                    axis=1)

    results_df = analyse_and_print_observations(df,"total",SEGMENT_CATEGORIES,NUMERATOR_COLUMN,DENOMINATOR_COLUMN,NUMERATOR_COLUMN_PERCENTAGE,FILTER_THRESHOLD)
    print("\n***************************************\n")
    results_df = analyse_and_print_observations(df,"average",SEGMENT_CATEGORIES,NUMERATOR_COLUMN,DENOMINATOR_COLUMN,NUMERATOR_COLUMN_PERCENTAGE,FILTER_THRESHOLD)

Read CSV
analysing for total metric
TOTAL purchases 42316463 (Up 247.67% from yesterday)
Users who are from australia: $4125434 (Up 212.58%  from yesterday)
Users who are from australia and are not VIP: $2040772 (Up 236.97%  from yesterday)
Users who are from australia and are VIP: $2084662 (Up 191.90%  from yesterday)
Users who are from brazil: $4101230 (Up 268.30%  from yesterday)
Users who are from brazil and are not VIP: $2021579 (Up 257.99%  from yesterday)
Users who are from brazil and are VIP: $2079651 (Up 278.90%  from yesterday)
Users who are from canada: $4163058 (Up 209.24%  from yesterday)
Users who are from canada and are not VIP: $1702953 (Up 236.50%  from yesterday)
Users who are from canada and are VIP: $2460104 (Up 192.83%  from yesterday)
Users who are from korea: $4349214 (Up 242.63%  from yesterday)
Users who are from korea and are not VIP: $2202163 (Up 238.26%  from yesterday)
Users who are from mexico: $3998506 (Up 245.37%  from yesterday)
Users who are from mexic

# Problem Statement

Take Home Test Instructions:

Product Requirements::
- Technology stack: Python, Pandas and/or SQL
- Dataset: comparative.ai take home test.csv

Product Definition:
Build an algorithm to look for user segments having significant metric changes today vs yesterday.

Metrics:
- Total purchased amount
- Average purchased amount per paying user (exclude users who did not pay)User segments:
Any combination of country and is_vip values.Examples:
- Users from Canada
- Users from Canada and are VIP
- Users who are not VIP

Significant metric change definition:



Need to meet all of the following conditions:
- Segment’s change absolute percentage is 2% greater than Overall’s change absolute percentage
- Segment’s change absolute value is greater than 2% of Overall’s change absolute value

Example output format:
```
  - Total purchased amount: $100 (up 10% from yesterday)
  - Users from Canada: $50 (down 20% from yesterday)
  - Users from Canada and are not VIP: $10 (up 12% from yesterday)
  - Users who are VIP: $30 (down %10 from yesterday)
  ...
  - Average purchased amount per paying user: $3.4 (up %20 from yesterday)
```
...Bonus points:
1. Group segments as much as possible. For example, instead of printing:
- Users from Canada
- Users who are VIP
- Users from Canada and are not VIP
- Users from Canada and are VIP

We’d like to see:
- Users from Canada
- Users from Canada and are not VIP
- Users from Canada and are VIP
- Users who are VIP
2. Parallelize your algorithm across multiple cores.Submission:


Please send us the Github repository link or zip file with your code within 3 days. We will evaluate the code on:
- Code Quality and Readibility
- Python, Pandas and/or SQL best practices